In [17]:
import requests
import os
from typing import Dict
from typing import Tuple
from typing import Optional


host_name = "www.deribit.com"
deribit_endpoint_v2 = f"https://{host_name}/api/v2"
public_url = f"{deribit_endpoint_v2}/public"
private_url = f"{deribit_endpoint_v2}/private"



def authorize_with_credentials(client_id: str, client_secret: str) -> Tuple[Dict[str, str], str]:
    print(f"connecting with {client_id}")
    headers = {"Content-Type": "application/json"}
    auth_params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials"
    }
    authentication = requests.get(f"{public_url}/auth", params=auth_params, headers=headers).json()
    headers["Authorization"] = f"Bearer {authentication['result']['access_token']}"
    refresh_token = authentication['result']['refresh_token']
    return headers, refresh_token

def authorize_refresh(refresh_token: str) -> Tuple[Dict[str, str], str]:
    print(f"refreshing authentication {refresh_token}")
    headers = {"Content-Type": "application/json"}
    auth_params = {
        "refresh_token": refresh_token,
        "grant_type": "refresh_token"
    }
    authentication = requests.get(f"{public_url}/auth", params=auth_params, headers=headers).json()
    headers["Authorization"] = f"Bearer {authentication['result']['access_token']}"
    new_refresh_token = authentication['result']['refresh_token']
    return headers, new_refresh_token

headers, refresh_token = authorize_with_credentials(client_id=os.getenv("DERIBIT_CLIENT_ID"), client_secret=os.getenv("DERIBIT_CLIENT_SECRET"))

connecting with 7XMV2uf-


In [18]:
def run_get(target: str, params: Dict[str, str], headers: Dict[str, str], refresh_token: str=None) -> Optional[Tuple[Dict[str, str], str]]:
    response = requests.get(target, params=params, headers=headers)
    if response.status_code >= 399:
        error = response.json()
        if error['code'] == 13009 and refresh_token:
            # re-run authorization
            new_headers, new_refresh_token = authorize_refresh(refresh_token)
            response = run_get(target, params, new_headers, new_refresh_token)
        else:
            return None

    return response.json()['result'], headers, refresh_token
    

account_summary = {}
for currency in ("BTC", "ETH", "USDT", "USDC"):
    account_summary_params = {"currency": currency}
    result, headers, refresh_token = run_get(f"{private_url}/get_account_summary", params=account_summary_params, headers=headers, refresh_token=refresh_token)
    account_summary[currency] = result

account_summary


{'BTC': {'margin_model': 'legacy_pm',
  'delta_total_map': {'btc_usd': -110.664037996},
  'options_vega': -14498.78968,
  'session_upl': 0.51353016,
  'projected_delta_total': -102.341496,
  'options_gamma': -0.00225,
  'options_theta_map': {'btc_usd': 3678.91562},
  'total_pl': -73.27017086,
  'projected_initial_margin': 44.74383869,
  'available_withdrawal_funds': 72.0200396,
  'deposit_address': 'bc1qw8rj57p23gwq70s87h49p8j8hs0gzy7xla4yxl',
  'limits': {'non_matching_engine': {'burst': 100, 'rate': 20},
   'matching_engine': {'burst': 30, 'rate': 10}},
  'options_delta': -110.59929,
  'options_pl': -73.22551958,
  'futures_session_upl': 0.00037959,
  'portfolio_margining_enabled': True,
  'delta_total': -102.341496,
  'maintenance_margin': 36.48778746,
  'cross_collateral_enabled': False,
  'futures_pl': -0.04465129,
  'session_rpl': 0.0,
  'options_value': -8.32254156,
  'projected_maintenance_margin': 36.48778746,
  'options_gamma_map': {'btc_usd': -0.00225},
  'options_session_rp